In [7]:
import bs4
import requests
import spacy
from spacy import displacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx
import pandas as pd

In [8]:
candidate_sentences = pd.read_csv(
    "/var/wd_smit/external_repos/knowledge-graph/data/output/kg/input_data.txt-out.csv"
)

In [9]:
df = pd.DataFrame(
    {
        "source": list(
            i.strip() for i in candidate_sentences[candidate_sentences.columns[0]]
        ),
        "edge": list(
            i.strip() for i in candidate_sentences[candidate_sentences.columns[1]]
        ),
        "target": list(
            i.strip() for i in candidate_sentences[candidate_sentences.columns[2]]
        ),
    }
)

In [10]:
df

,source,edge,target
0,Godfather,is,1972 American crime film
1,Godfather,is,1972 crime film
2,Mario Puzo,on,best-selling novel of same name
3,Godfather,is,American
4,It,stars,Marlon Brando
5,It,stars Marlon Brando as,leaders of fictional New York crime family
6,It,stars Marlon Brando as,leaders
7,story,chronicles family under,patriarch Vito Corleone
8,story,chronicles family under,patriarch Vito Corleone focusing on transforma...
9,story,chronicles family under,patriarch Vito Corleone focusing on transforma...


In [11]:
# create a directed-graph from a dataframe
G = nx.from_pandas_edgelist(
    df, "source", "target", edge_attr=True, create_using=nx.MultiDiGraph()
)

conversation_history = []

In [12]:
conversation_history = []


def return_context(noun):
    x = noun

    sentences = []

    if x in G.nodes:
        neighbor_nodes = list(G.neighbors(x))  # get the list of neighbors of node n
        neighbor_nodes.append(x)  # add node n to the list
        H = G.subgraph(neighbor_nodes)

        neighbor_edges = list(G.edges(x, data=True))

        for u, v, d in neighbor_edges:
            verb = d["edge"]  # get the verb from the edge label
            sentences.append(
                f"{u} {verb} {v}."
            )  # create a sentence with the source, verb,

    return sentences


def get_entities(sentence):
    # Parse the sentence using spaCy
    doc = nlp(sentence)

    # Print the text and label of each noun phrase
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.label_)

    lister = [i.text for i in doc.noun_chunks]

    for i in lister:
        lisst = i.split(" ")

        if "the" in lisst:
            lisst.remove("the")

        sentence = ""
        for x in lisst:
            sentence = sentence + str(x)

        i = x
    return lister